# **Capstone Project**

## Leading Indicators
| File Name | Type | Use |
|-----------|------|-----|
| `Consumer_Confidence_Index.csv` | Leading | Feature |
| `Business_Confidence_Index.csv` | Leading | Feature |
| `Initial_Claims.csv` | Leading | Feature |
| `Jobs_Added.csv` | Leading | Feature |
| `Housing_Starts.csv` | Leading | Feature |
| `Yield_Curve.csv` | Leading | Feature |
| `Federal_Funds_Rate.csv` | Leading (policy-reactive) | Feature |
| `Supply_Chain_Index.xls` | Leading | Feature |
| `Crude_Oil_Prices.csv` | Contextual | Feature |

In [7]:
import pandas as pd
import sqlite3

# Load CSV files into DataFrames
BCI_df = pd.read_csv('Leading/Business_Confidence_Index.csv')
CCI_df = pd.read_csv('Leading/Consumer_Confidence_Index.csv', engine='python', on_bad_lines='skip')
Oil_Prices_df = pd.read_csv('Leading/Crude_Oil_Prices.csv')
Interest_Rates_df = pd.read_csv('Leading/Interest_Rates.csv')
Yield_Curve_df = pd.read_csv('Leading/Yield_Curve.csv')
Housing_Starts_df = pd.read_csv('Leading/Housing_Starts.csv')
Jobless_Claims_df = pd.read_csv('Leading/Initial_Claims.csv')
Jobs_Added_df = pd.read_csv('Leading/Jobs_Added.csv')

# ---------------------------------------------------
# 🗃️ Load DataFrames into an SQLite in-memory database
# ---------------------------------------------------
conn = sqlite3.connect(':memory:')

BCI_df.to_sql('BusinessConfidence', conn, index=False, if_exists='replace')
CCI_df.to_sql('ConsumerConfidence', conn, index=False, if_exists='replace')
Oil_Prices_df.to_sql('OilPrices', conn, index=False, if_exists='replace')
Interest_Rates_df.to_sql('InterestRates', conn, index=False, if_exists='replace')
Yield_Curve_df.to_sql('YieldCurve', conn, index=False, if_exists='replace')
Housing_Starts_df.to_sql('HousingStarts', conn, index=False, if_exists='replace')
Jobless_Claims_df.to_sql('JoblessClaims', conn, index=False, if_exists='replace')
Jobs_Added_df.to_sql('JobsAdded', conn, index=False, if_exists='replace')

# ---------------------------
# 📊 SQL Query to Join Tables
# ---------------------------
sql_query = """
SELECT *
FROM InterestRates
LEFT JOIN BusinessConfidence ON InterestRates.observation_date = BusinessConfidence.observation_date
LEFT JOIN ConsumerConfidence ON InterestRates.observation_date = ConsumerConfidence.observation_date
LEFT JOIN OilPrices ON InterestRates.observation_date = OilPrices.observation_date
LEFT JOIN YieldCurve ON InterestRates.observation_date = YieldCurve.observation_date
LEFT JOIN HousingStarts ON InterestRates.observation_date = HousingStarts.observation_date
LEFT JOIN JoblessClaims ON InterestRates.observation_date = JoblessClaims.observation_date
LEFT JOIN JobsAdded ON InterestRates.observation_date = JobsAdded.observation_date
WHERE InterestRates.observation_date >= '1978-01-01'
"""

# Execute the query and load results into a DataFrame
Leading_Indicators_df = pd.read_sql_query(sql_query, conn)

# Save the merged DataFrame to a new CSV file
csv_filename = "Leading/Leading_Indicators_df.csv"
Leading_Indicators_df.to_csv(csv_filename, index=False)

# Close the connection
conn.close()

# Output the final DataFrame
Leading_Indicators_df


,observation_date,DFF,observation_date,BSCICP02USM460S,observation_date,UMCSENT,observation_date,WTISPLC,observation_date,T10Y2Y,observation_date,HOUST,observation_date,ICSA,observation_date,PAYEMS
0,1978-01-01,6.76,1978-01-01,14.8,1978-01-01,83.7,1978-01-01,14.85,1978-01-01,0.47,1978-01-01,1718,1978-01-01,361083,1978-01-01,84594
1,1978-04-01,7.28,1978-04-01,15.4,1978-04-01,81.6,1978-04-01,14.85,1978-04-01,0.41,1978-04-01,2197,1978-04-01,325769,1978-04-01,86162
2,1978-07-01,8.09,1978-07-01,24.4,1978-07-01,82.4,1978-07-01,14.85,1978-07-01,0.15,1978-07-01,2092,1978-07-01,339929,1978-07-01,87204
3,1978-10-01,9.58,1978-10-01,20.2,1978-10-01,79.3,1978-10-01,14.85,1978-10-01,-0.21,1978-10-01,1981,1978-10-01,339462,1978-10-01,87956
4,1979-01-01,10.07,1979-01-01,17.0,1979-01-01,72.1,1979-01-01,14.85,1979-01-01,-0.76,1979-01-01,1630,1979-01-01,358000,1979-01-01,88808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,2024-01-01,5.33,2024-01-01,-1.8,2024-01-01,79.0,2024-01-01,74.15,2024-01-01,-0.26,2024-01-01,1376,2024-01-01,212154,2024-01-01,157049
186,2024-04-01,5.33,2024-04-01,-2.4,2024-04-01,77.2,2024-04-01,85.35,2024-04-01,-0.33,2024-04-01,1377,2024-04-01,223615,2024-04-01,157635
187,2024-07-01,5.26,2024-07-01,-6.0,2024-07-01,66.4,2024-07-01,81.80,2024-07-01,-0.25,2024-07-01,1262,2024-07-01,230769,2024-07-01,158003
188,2024-10-01,4.65,2024-10-01,-6.2,2024-10-01,70.5,2024-10-01,71.99,2024-10-01,0.12,2024-10-01,1344,2024-10-01,225462,2024-10-01,158358


## Lagging Indicators
| File Name | Type | Use |
|-----------|------|-----|
| `PPI_Inflation_Rate.csv` | Lagging | Feature |
| `CPI_Inflation_Rate.csv` | Lagging | Feature |
| `Unemployment_Rate.csv` | Lagging | Feature |
| `Labor_Force_Participation.csv` | Lagging | Feature |
| `Real_Gross_Domestic_Product.csv` | Lagging | **Label source** |
| `Deficit_Percent_GDP.csv` | Lagging | Feature (used in derived fiscal stress index) |


In [8]:
import pandas as pd
import sqlite3

# Load CSV files into DataFrames
CPI_Inflation_df = pd.read_csv('Lagging/CPI_Inflation_Rate.csv')
Labor_Force_df = pd.read_csv('Lagging/Labor_Force_Participation.csv')
PPI_Inflation_df = pd.read_csv('Lagging/PPI_Inflation_Rate.csv')
GDP_df = pd.read_csv('Lagging/Real_Gross_Domestic_Product.csv')
Unemployment_df = pd.read_csv('Lagging/Unemployment_Rate.csv')

# Set up in-memory SQLite connection
conn = sqlite3.connect(':memory:')

# Load data into SQL tables
CPI_Inflation_df.to_sql('CPI', conn, index=False, if_exists='replace')
Labor_Force_df.to_sql('Labor_Force', conn, index=False, if_exists='replace')
PPI_Inflation_df.to_sql('PPI', conn, index=False, if_exists='replace')
GDP_df.to_sql('GDP', conn, index=False, if_exists='replace')
Unemployment_df.to_sql('Unemployment', conn, index=False, if_exists='replace')

# SQL query to join all lagging indicator tables on observation_date
sql_query = """
SELECT *
FROM GDP
LEFT JOIN CPI ON GDP.observation_date = CPI.observation_date
LEFT JOIN Labor_Force ON GDP.observation_date = Labor_Force.observation_date
LEFT JOIN PPI ON GDP.observation_date = PPI.observation_date
LEFT JOIN Unemployment ON GDP.observation_date = Unemployment.observation_date
WHERE GDP.observation_date >= '1976-01-01'
"""

# Execute query and load result into DataFrame
Lagging_Indicators_df = pd.read_sql_query(sql_query, conn)

# Save to CSV
Lagging_Indicators_df.to_csv("Lagging/Lagging_Indicators_df.csv", index=False)

# Close the database connection
conn.close()

# Display the final joined DataFrame
Lagging_Indicators_df.head()



,observation_date,GDPC1,observation_date,CPIAUCSL,observation_date,CIVPART,observation_date,PPIACO,observation_date,UNRATE
0,1976-01-01,6323.649,1976-01-01,55.8,1976-01-01,61.3,1976-01-01,59.9,1976-01-01,7.9
1,1976-04-01,6370.025,1976-04-01,56.1,1976-04-01,61.6,1976-04-01,60.6,1976-04-01,7.7
2,1976-07-01,6404.895,1976-07-01,57.0,1976-07-01,61.8,1976-07-01,61.6,1976-07-01,7.8
3,1976-10-01,6451.177,1976-10-01,57.9,1976-10-01,61.6,1976-10-01,61.9,1976-10-01,7.7
4,1977-01-01,6527.703,1977-01-01,58.7,1977-01-01,61.6,1977-01-01,62.8,1977-01-01,7.5


## Coincident Indicators

| File Name | Type | Use |
|-----------|------|-----|
| `Capacity_Utilization_Index.csv` | Coincident / Leading | Feature + Label Logic |
| `Industrial_Production_Index.csv` | Coincident | Feature |


In [9]:
import pandas as pd
import sqlite3

# Load CSV files into DataFrames
Capacity_df = pd.read_csv('Coincident/Capacity_Utilization_Index.csv')
Industrial_Production_df = pd.read_csv('Coincident/Industrial_Production_Index.csv', engine='python', on_bad_lines='skip')

# ---------------------------------------------------
# 🗃️ Load DataFrames into an SQLite in-memory database
# ---------------------------------------------------
conn = sqlite3.connect(':memory:')

Capacity_df.to_sql('Capacity', conn, index=False, if_exists='replace')
Industrial_Production_df.to_sql('Industrial', conn, index=False, if_exists='replace')


# ---------------------------
# 📊 SQL Query to Join Tables
# ---------------------------
sql_query = """
SELECT *
FROM Capacity
LEFT JOIN Industrial ON Capacity.observation_date = Industrial.observation_date
WHERE Capacity.observation_date >= '1978-01-01'
"""

# Execute the query and load results into a DataFrame
Coincident_Indicators_df = pd.read_sql_query(sql_query, conn)

# Save the merged DataFrame to a new CSV file
csv_filename = "Coincident/Coincident_Indicators_df.csv"
Coincident_Indicators_df.to_csv(csv_filename, index=False)

# Close the connection
conn.close()

# Provide the final DataFrame
Coincident_Indicators_df


,observation_date,TCU,observation_date,INDPRO
0,1978-01-01,82.8849,1978-01-01,47.7512
1,1978-04-01,85.5183,1978-04-01,49.9074
2,1978-07-01,85.5628,1978-07-01,50.4371
3,1978-10-01,86.3582,1978-10-01,51.1523
4,1979-01-01,86.0650,1979-01-01,51.5250
...,...,...,...,...
183,2023-10-01,78.2766,2023-10-01,102.5781
184,2024-01-01,77.7036,2024-01-01,101.4830
185,2024-04-01,77.9731,2024-04-01,102.3568
186,2024-07-01,77.6310,2024-07-01,102.5192


## Renaming Columns

In [10]:
# Remove duplicate columns
Final_Lagging_Indicators_df = Lagging_Indicators_df.loc[:, ~Lagging_Indicators_df.columns.duplicated()].copy()

# Rename columns
Final_Lagging_Indicators_df.columns = [
    'Date',
    'GDP_Growth_Rate',
    'Change_in_CPI',
    'Labor_Force_Change',
    'Change_in_PPI',
    'Unemployment_Rate_Change'
]

# Apply transformations
Final_Lagging_Indicators_df['GDP_Growth_Rate'] = Final_Lagging_Indicators_df['GDP_Growth_Rate'].pct_change().round(2) * 100
Final_Lagging_Indicators_df['Change_in_CPI'] = Final_Lagging_Indicators_df['Change_in_CPI'].diff().round(2)
Final_Lagging_Indicators_df['Labor_Force_Change'] = Final_Lagging_Indicators_df['Labor_Force_Change'].diff().round(2)
Final_Lagging_Indicators_df['Change_in_PPI'] = Final_Lagging_Indicators_df['Change_in_PPI'].diff().round(2)
Final_Lagging_Indicators_df['Unemployment_Rate_Change'] = Final_Lagging_Indicators_df['Unemployment_Rate_Change'].diff().round(2)

# Preview
Final_Lagging_Indicators_df.head()


,Date,GDP_Growth_Rate,Change_in_CPI,Labor_Force_Change,Change_in_PPI,Unemployment_Rate_Change
0,1976-01-01,NaN,NaN,NaN,NaN,NaN
1,1976-04-01,1.0,0.3,0.3,0.7,-0.2
2,1976-07-01,1.0,0.9,0.2,1.0,0.1
3,1976-10-01,1.0,0.9,-0.2,0.3,-0.1
4,1977-01-01,1.0,0.8,0.0,0.9,-0.2


In [11]:
# Remove duplicate columns
Final_Leading_Indicators_df = Leading_Indicators_df.loc[:, ~Leading_Indicators_df.columns.duplicated()].copy()

# Rename columns
Final_Leading_Indicators_df.columns = [
    'Date',
    'Fed_Funds_Rate_Change',
    'Business_Sentiment_Growth_Rate',
    'Consumer_Sentiment_Growth_Rate',
    'Oil_Price_Change',
    'Yield_Curve_Change',
    'Housing_Starts_Change',
    'Jobless_Claims_Growth_Rate',
    'Job_Growth_Rate'
]

# Transformations
Final_Leading_Indicators_df['Fed_Funds_Rate_Change'] = Final_Leading_Indicators_df['Fed_Funds_Rate_Change'].diff().round(2)
Final_Leading_Indicators_df['Business_Sentiment_Growth_Rate'] = Final_Leading_Indicators_df['Business_Sentiment_Growth_Rate'].pct_change().round(2) * 100
Final_Leading_Indicators_df['Consumer_Sentiment_Growth_Rate'] = Final_Leading_Indicators_df['Consumer_Sentiment_Growth_Rate'].pct_change().round(2) * 100
Final_Leading_Indicators_df['Oil_Price_Change'] = Final_Leading_Indicators_df['Oil_Price_Change'].pct_change().round(2) * 100
Final_Leading_Indicators_df['Yield_Curve_Change'] = Final_Leading_Indicators_df['Yield_Curve_Change'].diff().round(2)
Final_Leading_Indicators_df['Housing_Starts_Change'] = Final_Leading_Indicators_df['Housing_Starts_Change'].pct_change().round(2) * 100
Final_Leading_Indicators_df['Jobless_Claims_Growth_Rate'] = Final_Leading_Indicators_df['Jobless_Claims_Growth_Rate'].pct_change().round(2) * 100
Final_Leading_Indicators_df['Job_Growth_Rate'] = Final_Leading_Indicators_df['Job_Growth_Rate'].pct_change().round(2) * 100

# Preview
Final_Leading_Indicators_df.head()


,Date,Fed_Funds_Rate_Change,Business_Sentiment_Growth_Rate,Consumer_Sentiment_Growth_Rate,Oil_Price_Change,Yield_Curve_Change,Housing_Starts_Change,Jobless_Claims_Growth_Rate,Job_Growth_Rate
0,1978-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1978-04-01,0.52,4.0,-3.0,0.0,-0.06,28.0,-10.0,2.0
2,1978-07-01,0.81,58.0,1.0,0.0,-0.26,-5.0,4.0,1.0
3,1978-10-01,1.49,-17.0,-4.0,0.0,-0.36,-5.0,-0.0,1.0
4,1979-01-01,0.49,-16.0,-9.0,0.0,-0.55,-18.0,5.0,1.0


In [12]:
import pandas as pd

# Remove duplicated columns from Coincident_Indicators_df
Final_Coincident_Indicators_df = Coincident_Indicators_df.loc[:, ~Coincident_Indicators_df.columns.duplicated()]

# Rename columns to growth rate versions
Final_Coincident_Indicators_df.columns = [
    'Date',
    'Capacity_Utilization_Growth_Rate',
    'Industrial_Production_Growth_Rate'
]

# Calculate percent change (growth rate) for Capacity Utilization
Final_Coincident_Indicators_df['Capacity_Utilization_Growth_Rate'] = (
    Final_Coincident_Indicators_df['Capacity_Utilization_Growth_Rate'].pct_change() * 100
).round(2)

# Calculate percent change (growth rate) for Industrial Production
Final_Coincident_Indicators_df['Industrial_Production_Growth_Rate'] = (
    Final_Coincident_Indicators_df['Industrial_Production_Growth_Rate'].pct_change() * 100
).round(2)

# Display the resulting DataFrame
Final_Coincident_Indicators_df.head()


C:\Users\maktr\AppData\Local\Temp\ipykernel_18208\1993585130.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_Coincident_Indicators_df['Capacity_Utilization_Growth_Rate'] = (
C:\Users\maktr\AppData\Local\Temp\ipykernel_18208\1993585130.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_Coincident_Indicators_df['Industrial_Production_Growth_Rate'] = (


,Date,Capacity_Utilization_Growth_Rate,Industrial_Production_Growth_Rate
0,1978-01-01,NaN,NaN
1,1978-04-01,3.18,4.52
2,1978-07-01,0.05,1.06
3,1978-10-01,0.93,1.42
4,1979-01-01,-0.34,0.73


### 1980 Recession (Start= January 1980)

In [13]:
import pandas as pd
import sqlite3

# Create in-memory SQLite connection
conn = sqlite3.connect(':memory:')

# Assume these DataFrames are already loaded:
# Final_Coincident_Indicators_df, Final_Leading_Indicators_df, Final_Lagging_Indicators_df

# Save them to the SQLite database
Final_Coincident_Indicators_df.to_sql('Coincident', conn, index=False, if_exists='replace')
Final_Leading_Indicators_df.to_sql('Leading', conn, index=False, if_exists='replace')
Final_Lagging_Indicators_df.to_sql('Lagging', conn, index=False, if_exists='replace')

# Proper SQL query
sql_query = """
SELECT *
FROM Leading LD
JOIN Lagging LG ON LD.date = LG.date
JOIN Coincident C ON LD.date = C.date
WHERE LD.date >= '1979-01-01'
AND LD.date <= '1981-01-01'
"""


# Run the query
Pre_1980_Recession = pd.read_sql_query(sql_query, conn)

# Save to CSV
csv_filename = "Pre_1980_Recession.csv"
Pre_1980_Recession.to_csv(csv_filename, index=False)

# Close DB connection
conn.close()

# Show final DataFrame
Pre_1980_Recession


,Date,Fed_Funds_Rate_Change,Business_Sentiment_Growth_Rate,Consumer_Sentiment_Growth_Rate,Oil_Price_Change,Yield_Curve_Change,Housing_Starts_Change,Jobless_Claims_Growth_Rate,Job_Growth_Rate,Date,GDP_Growth_Rate,Change_in_CPI,Labor_Force_Change,Change_in_PPI,Unemployment_Rate_Change,Date,Capacity_Utilization_Growth_Rate,Industrial_Production_Growth_Rate
0,1979-01-01,0.49,-16.0,-9.0,0.0,-0.55,-18.0,5.0,1.0,1979-01-01,0.0,1.4,0.3,2.0,0.1,1979-01-01,-0.34,0.73
1,1979-04-01,0.11,-27.0,-8.0,7.0,0.16,7.0,4.0,1.0,1979-04-01,0.0,2.1,-0.1,3.1,-0.1,1979-04-01,-1.03,-0.39
2,1979-07-01,0.76,-79.0,-8.0,37.0,0.41,1.0,3.0,1.0,1979-07-01,1.0,2.4,0.1,2.3,-0.1,1979-07-01,-1.05,0.74
3,1979-10-01,2.64,-177.0,3.0,33.0,-1.00,-4.0,8.0,0.0,1979-10-01,0.0,2.2,0.1,2.9,0.3,1979-10-01,-0.29,0.03
4,1980-01-01,1.49,280.0,8.0,12.0,0.49,-20.0,2.0,0.0,1980-01-01,0.0,2.8,0.3,3.1,0.3,1980-01-01,-0.18,0.44
5,1980-04-01,-2.40,232.0,-21.0,22.0,-0.33,-22.0,39.0,0.0,1980-04-01,-2.0,2.9,-0.2,2.6,0.6,1980-04-01,-4.76,-2.36
6,1980-07-01,-2.85,19.0,18.0,0.0,2.25,21.0,-11.0,-1.0,1980-07-01,0.0,1.7,0.0,2.5,0.9,1980-07-01,-2.17,-4.39
7,1980-10-01,6.03,-137.0,20.0,-9.0,-1.56,20.0,-20.0,1.0,1980-10-01,2.0,2.1,-0.1,2.5,-0.3,1980-10-01,3.26,3.29
8,1981-01-01,0.75,-115.0,-5.0,6.0,-0.35,2.0,-1.0,1.0,1981-01-01,2.0,2.5,0.2,2.4,0.0,1981-01-01,-0.40,1.69


### 1981 Recession (Start= July 1981)

In [14]:
import pandas as pd
import sqlite3

# Create in-memory SQLite connection
conn = sqlite3.connect(':memory:')

# Assume these DataFrames are already loaded:
# Final_Coincident_Indicators_df, Final_Leading_Indicators_df, Final_Lagging_Indicators_df

# Save them to the SQLite database
Final_Coincident_Indicators_df.to_sql('Coincident', conn, index=False, if_exists='replace')
Final_Leading_Indicators_df.to_sql('Leading', conn, index=False, if_exists='replace')
Final_Lagging_Indicators_df.to_sql('Lagging', conn, index=False, if_exists='replace')

# Proper SQL query
sql_query = """
SELECT *
FROM Leading LD
JOIN Lagging LG ON LD.date = LG.date
JOIN Coincident C ON LD.date = C.date
WHERE LD.date >= '1980-07-01'
AND LD.date <= '1982-07-01'
"""


# Run the query
Pre_1981_Recession = pd.read_sql_query(sql_query, conn)

# Save to CSV
csv_filename = "Pre_1981_Recession.csv"
Pre_1981_Recession.to_csv(csv_filename, index=False)

# Close DB connection
conn.close()

# Show final DataFrame
Pre_1981_Recession


,Date,Fed_Funds_Rate_Change,Business_Sentiment_Growth_Rate,Consumer_Sentiment_Growth_Rate,Oil_Price_Change,Yield_Curve_Change,Housing_Starts_Change,Jobless_Claims_Growth_Rate,Job_Growth_Rate,Date,GDP_Growth_Rate,Change_in_CPI,Labor_Force_Change,Change_in_PPI,Unemployment_Rate_Change,Date,Capacity_Utilization_Growth_Rate,Industrial_Production_Growth_Rate
0,1980-07-01,-2.85,19.0,18.0,0.0,2.25,21.0,-11.0,-1.0,1980-07-01,0.0,1.7,0.0,2.5,0.9,1980-07-01,-2.17,-4.39
1,1980-10-01,6.03,-137.0,20.0,-9.0,-1.56,20.0,-20.0,1.0,1980-10-01,2.0,2.1,-0.1,2.5,-0.3,1980-10-01,3.26,3.29
2,1981-01-01,0.75,-115.0,-5.0,6.0,-0.35,2.0,-1.0,1.0,1981-01-01,2.0,2.5,0.2,2.4,0.0,1981-01-01,-0.40,1.69
3,1981-04-01,1.19,-300.0,1.0,0.0,0.22,-12.0,1.0,0.0,1981-04-01,-1.0,1.9,0.3,2.8,-0.3,1981-04-01,-0.36,-0.33
4,1981-07-01,-0.20,-306.0,2.0,-5.0,-0.60,-23.0,8.0,0.0,1981-07-01,1.0,2.4,-0.4,1.0,0.0,1981-07-01,0.22,1.75
5,1981-10-01,-4.00,203.0,-5.0,-3.0,0.68,-16.0,14.0,0.0,1981-10-01,-1.0,1.9,0.0,-0.1,0.7,1981-10-01,-2.94,-1.37
6,1982-01-01,0.62,18.0,1.0,-3.0,0.42,-3.0,5.0,-1.0,1982-01-01,-2.0,1.0,-0.1,0.8,0.7,1982-01-01,-2.70,-4.24
7,1982-04-01,0.30,3.0,-8.0,-1.0,-0.36,9.0,9.0,0.0,1982-04-01,0.0,0.6,0.2,-0.1,0.7,1982-04-01,-1.81,0.42
8,1982-07-01,-3.50,-5.0,0.0,2.0,0.48,27.0,5.0,-1.0,1982-07-01,0.0,2.5,0.1,0.8,0.5,1982-07-01,-1.88,-1.25


### 1990 Recession (Start= July 1990)

In [15]:
import pandas as pd
import sqlite3

# Create in-memory SQLite connection
conn = sqlite3.connect(':memory:')

# Assume these DataFrames are already loaded:
# Final_Coincident_Indicators_df, Final_Leading_Indicators_df, Final_Lagging_Indicators_df

# Save them to the SQLite database
Final_Coincident_Indicators_df.to_sql('Coincident', conn, index=False, if_exists='replace')
Final_Leading_Indicators_df.to_sql('Leading', conn, index=False, if_exists='replace')
Final_Lagging_Indicators_df.to_sql('Lagging', conn, index=False, if_exists='replace')

# Proper SQL query
sql_query = """
SELECT *
FROM Leading LD
JOIN Lagging LG ON LD.date = LG.date
JOIN Coincident C ON LD.date = C.date
WHERE LD.date >= '1989-07-01'
AND LD.date <= '1991-07-01'
"""


# Run the query
Pre_1990_Recession = pd.read_sql_query(sql_query, conn)

# Save to CSV
csv_filename = "Pre_1990_Recession.csv"
Pre_1990_Recession.to_csv(csv_filename, index=False)

# Close DB connection
conn.close()

# Show final DataFrame
Pre_1990_Recession


,Date,Fed_Funds_Rate_Change,Business_Sentiment_Growth_Rate,Consumer_Sentiment_Growth_Rate,Oil_Price_Change,Yield_Curve_Change,Housing_Starts_Change,Jobless_Claims_Growth_Rate,Job_Growth_Rate,Date,GDP_Growth_Rate,Change_in_CPI,Labor_Force_Change,Change_in_PPI,Unemployment_Rate_Change,Date,Capacity_Utilization_Growth_Rate,Industrial_Production_Growth_Rate
0,1989-07-01,-0.65,-286.0,1.0,-7.0,0.48,7.0,5.0,0.0,1989-07-01,1.0,1.4,0.1,0.5,0.0,1989-07-01,-1.27,-1.51
1,1989-10-01,-0.47,-22.0,2.0,2.0,-0.17,-1.0,4.0,0.0,1989-10-01,0.0,0.9,0.0,0.0,0.1,1989-10-01,-0.34,0.51
2,1990-01-01,-0.36,-13.0,-1.0,13.0,0.09,10.0,1.0,1.0,1990-01-01,1.0,2.1,0.3,2.1,0.1,1990-01-01,0.00,0.38
3,1990-04-01,-0.01,-91.0,1.0,-18.0,-0.06,-20.0,2.0,0.0,1990-04-01,0.0,1.4,-0.2,-0.8,0.0,1990-04-01,0.03,1.16
4,1990-04-01,0.00,-100.0,0.0,0.0,0.00,0.0,0.0,0.0,1990-04-01,0.0,1.4,-0.2,-0.8,0.0,1990-04-01,0.03,1.16
5,1990-07-01,-0.08,-inf,-6.0,0.0,0.25,-6.0,6.0,0.0,1990-07-01,0.0,1.6,-0.1,0.4,0.1,1990-07-01,-0.16,0.47
6,1990-10-01,-0.42,100.0,-28.0,93.0,0.53,-13.0,16.0,0.0,1990-10-01,-1.0,2.9,-0.1,6.3,0.4,1990-10-01,-1.97,-0.23
7,1991-01-01,-1.31,59.0,5.0,-31.0,0.13,-21.0,7.0,0.0,1991-01-01,0.0,1.3,-0.2,-1.8,0.5,1991-01-01,-2.20,-2.26
8,1991-04-01,-0.57,-33.0,22.0,-17.0,0.12,25.0,-6.0,-1.0,1991-04-01,1.0,0.4,0.2,-3.0,0.3,1991-04-01,0.31,-1.03
9,1991-07-01,-0.21,-108.0,1.0,3.0,0.27,6.0,-5.0,0.0,1991-07-01,1.0,1.1,-0.3,0.1,0.1,1991-07-01,1.15,2.10


### 2001 Recession (Start= March 2001)

In [16]:
import pandas as pd
import sqlite3

# Create in-memory SQLite connection
conn = sqlite3.connect(':memory:')

# Assume these DataFrames are already loaded:
# Final_Coincident_Indicators_df, Final_Leading_Indicators_df, Final_Lagging_Indicators_df

# Save them to the SQLite database
Final_Coincident_Indicators_df.to_sql('Coincident', conn, index=False, if_exists='replace')
Final_Leading_Indicators_df.to_sql('Leading', conn, index=False, if_exists='replace')
Final_Lagging_Indicators_df.to_sql('Lagging', conn, index=False, if_exists='replace')

# Proper SQL query
sql_query = """
SELECT *
FROM Leading LD
JOIN Lagging LG ON LD.date = LG.date
JOIN Coincident C ON LD.date = C.date
WHERE LD.date >= '2000-04-01'
AND LD.date <= '2002-04-01'
"""


# Run the query
Pre_2001_Recession = pd.read_sql_query(sql_query, conn)

# Save to CSV
csv_filename = "Pre_1990_Recession.csv"
Pre_2001_Recession.to_csv(csv_filename, index=False)

# Close DB connection
conn.close()

# Show final DataFrame
Pre_2001_Recession

,Date,Fed_Funds_Rate_Change,Business_Sentiment_Growth_Rate,Consumer_Sentiment_Growth_Rate,Oil_Price_Change,Yield_Curve_Change,Housing_Starts_Change,Jobless_Claims_Growth_Rate,Job_Growth_Rate,Date,GDP_Growth_Rate,Change_in_CPI,Labor_Force_Change,Change_in_PPI,Unemployment_Rate_Change,Date,Capacity_Utilization_Growth_Rate,Industrial_Production_Growth_Rate
0,2000-04-01,0.59,-30.0,-3.0,-5.0,-0.63,-1.0,-2.0,1.0,2000-04-01,2.0,1.6,0.0,2.4,-0.2,2000-04-01,0.21,1.37
1,2000-07-01,0.25,-47.0,-1.0,16.0,0.12,-10.0,8.0,0.0,2000-07-01,0.0,1.8,-0.4,3.0,0.2,2000-07-01,-1.06,0.18
2,2000-10-01,-0.05,-152.0,-2.0,11.0,0.12,6.0,9.0,0.0,2000-10-01,1.0,1.2,-0.1,1.7,-0.1,2000-10-01,-1.25,-0.21
3,2001-01-01,-0.87,562.0,-10.0,-11.0,0.57,3.0,12.0,0.0,2001-01-01,0.0,1.7,0.4,4.6,0.3,2001-01-01,-2.19,-0.81
4,2001-04-01,-1.27,-20.0,-7.0,-7.0,0.51,3.0,8.0,0.0,2001-04-01,1.0,0.8,-0.3,-3.6,0.2,2001-04-01,-2.16,-1.25
5,2001-07-01,-0.83,-23.0,5.0,-4.0,0.29,1.0,4.0,0.0,2001-07-01,0.0,1.0,-0.1,-3.0,0.2,2001-07-01,-2.15,-1.66
6,2001-10-01,-1.37,79.0,-10.0,-16.0,0.64,-8.0,8.0,-1.0,2001-10-01,0.0,0.2,-0.1,-3.1,0.7,2001-10-01,-1.75,-0.93
7,2002-01-01,-0.40,-91.0,12.0,-11.0,0.17,10.0,-8.0,0.0,2002-01-01,1.0,0.1,-0.2,-1.8,0.4,2002-01-01,0.22,0.07
8,2002-04-01,0.02,-589.0,0.0,34.0,-0.22,-6.0,0.0,0.0,2002-04-01,1.0,1.6,0.2,2.3,0.2,2002-04-01,1.25,1.23


### 2008 Recession (Start= December 2007)

In [17]:
import pandas as pd
import sqlite3

# Create in-memory SQLite connection
conn = sqlite3.connect(':memory:')

# Assume these DataFrames are already loaded:
# Final_Coincident_Indicators_df, Final_Leading_Indicators_df, Final_Lagging_Indicators_df

# Save them to the SQLite database
Final_Coincident_Indicators_df.to_sql('Coincident', conn, index=False, if_exists='replace')
Final_Leading_Indicators_df.to_sql('Leading', conn, index=False, if_exists='replace')
Final_Lagging_Indicators_df.to_sql('Lagging', conn, index=False, if_exists='replace')

# Proper SQL query
sql_query = """
SELECT *
FROM Leading LD
JOIN Lagging LG ON LD.date = LG.date
JOIN Coincident C ON LD.date = C.date
WHERE LD.date >= '2007-01-01'
AND LD.date <= '2009-01-01'
"""


# Run the query
Pre_2008_Recession = pd.read_sql_query(sql_query, conn)

# Save to CSV
csv_filename = "Pre_2008_Recession.csv"
Pre_2008_Recession.to_csv(csv_filename, index=False)

# Close DB connection
conn.close()

# Show final DataFrame
Pre_2008_Recession

,Date,Fed_Funds_Rate_Change,Business_Sentiment_Growth_Rate,Consumer_Sentiment_Growth_Rate,Oil_Price_Change,Yield_Curve_Change,Housing_Starts_Change,Jobless_Claims_Growth_Rate,Job_Growth_Rate,Date,GDP_Growth_Rate,Change_in_CPI,Labor_Force_Change,Change_in_PPI,Unemployment_Rate_Change,Date,Capacity_Utilization_Growth_Rate,Industrial_Production_Growth_Rate
0,2007-01-01,0.01,-164.0,4.0,-7.0,-0.05,-5.0,-2.0,0.0,2007-01-01,0.0,1.54,0.2,1.8,0.2,2007-01-01,0.30,0.64
1,2007-04-01,0.00,-500.0,-10.0,17.0,0.15,6.0,-1.0,0.0,2007-04-01,1.0,2.47,-0.5,7.4,-0.1,2007-04-01,0.53,1.82
2,2007-07-01,-0.18,-18.0,4.0,16.0,0.16,-9.0,1.0,0.0,2007-07-01,1.0,1.70,0.1,3.7,0.2,2007-07-01,-0.29,-0.08
3,2007-10-01,-0.57,-83.0,-11.0,16.0,0.37,-7.0,7.0,0.0,2007-10-01,1.0,1.59,-0.2,-0.4,0.0,2007-10-01,0.27,0.15
4,2008-01-01,-1.32,75.0,-3.0,8.0,0.71,-14.0,4.0,0.0,2008-01-01,0.0,2.98,0.4,6.3,0.3,2008-01-01,-0.07,0.50
5,2008-04-01,-1.10,-300.0,-20.0,21.0,0.36,-7.0,5.0,0.0,2008-04-01,1.0,1.77,-0.3,9.9,0.0,2008-04-01,-1.31,-1.36
6,2008-07-01,-0.14,-100.0,-2.0,19.0,-0.20,-9.0,17.0,0.0,2008-07-01,-1.0,5.07,0.2,14.6,0.8,2008-07-01,-3.34,-1.33
7,2008-10-01,-1.43,-inf,-6.0,-43.0,0.77,-16.0,20.0,-1.0,2008-10-01,-2.0,-2.02,-0.1,-19.1,0.7,2008-10-01,-4.59,-4.95
8,2009-01-01,-0.33,30.0,6.0,-46.0,-0.49,-37.0,19.0,-2.0,2009-01-01,-1.0,-5.06,-0.3,-15.2,1.3,2009-01-01,-6.03,-6.48


In [18]:
import pandas as pd
import sqlite3

# Create in-memory SQLite connection
conn = sqlite3.connect(':memory:')

# Assume these DataFrames are already loaded:
# Final_Coincident_Indicators_df, Final_Leading_Indicators_df, Final_Lagging_Indicators_df

# Save them to the SQLite database
Final_Coincident_Indicators_df.to_sql('Coincident', conn, index=False, if_exists='replace')
Final_Leading_Indicators_df.to_sql('Leading', conn, index=False, if_exists='replace')
Final_Lagging_Indicators_df.to_sql('Lagging', conn, index=False, if_exists='replace')

# Proper SQL query
sql_query = """
SELECT *
FROM Leading LD
JOIN Lagging LG ON LD.date = LG.date
JOIN Coincident C ON LD.date = C.date
WHERE LD.date >= '1979-01-01'
AND LD.date <= '1980-01-01'
"""


# Run the query
Pre_1980_Recession = pd.read_sql_query(sql_query, conn)

# Save to CSV
csv_filename = "Pre_1980_Recession.csv"
Pre_1980_Recession.to_csv(csv_filename, index=False)

# Close DB connection
conn.close()

# Show final DataFrame
Pre_1980_Recession


,Date,Fed_Funds_Rate_Change,Business_Sentiment_Growth_Rate,Consumer_Sentiment_Growth_Rate,Oil_Price_Change,Yield_Curve_Change,Housing_Starts_Change,Jobless_Claims_Growth_Rate,Job_Growth_Rate,Date,GDP_Growth_Rate,Change_in_CPI,Labor_Force_Change,Change_in_PPI,Unemployment_Rate_Change,Date,Capacity_Utilization_Growth_Rate,Industrial_Production_Growth_Rate
0,1979-01-01,0.49,-16.0,-9.0,0.0,-0.55,-18.0,5.0,1.0,1979-01-01,0.0,1.4,0.3,2.0,0.1,1979-01-01,-0.34,0.73
1,1979-04-01,0.11,-27.0,-8.0,7.0,0.16,7.0,4.0,1.0,1979-04-01,0.0,2.1,-0.1,3.1,-0.1,1979-04-01,-1.03,-0.39
2,1979-07-01,0.76,-79.0,-8.0,37.0,0.41,1.0,3.0,1.0,1979-07-01,1.0,2.4,0.1,2.3,-0.1,1979-07-01,-1.05,0.74
3,1979-10-01,2.64,-177.0,3.0,33.0,-1.00,-4.0,8.0,0.0,1979-10-01,0.0,2.2,0.1,2.9,0.3,1979-10-01,-0.29,0.03
4,1980-01-01,1.49,280.0,8.0,12.0,0.49,-20.0,2.0,0.0,1980-01-01,0.0,2.8,0.3,3.1,0.3,1980-01-01,-0.18,0.44


### 1981 Recession (Start= July 1981)

In [19]:
import pandas as pd
import sqlite3

# Create in-memory SQLite connection
conn = sqlite3.connect(':memory:')

# Assume these DataFrames are already loaded:
# Final_Coincident_Indicators_df, Final_Leading_Indicators_df, Final_Lagging_Indicators_df

# Save them to the SQLite database
Final_Coincident_Indicators_df.to_sql('Coincident', conn, index=False, if_exists='replace')
Final_Leading_Indicators_df.to_sql('Leading', conn, index=False, if_exists='replace')
Final_Lagging_Indicators_df.to_sql('Lagging', conn, index=False, if_exists='replace')

# Proper SQL query
sql_query = """
SELECT *
FROM Leading LD
JOIN Lagging LG ON LD.date = LG.date
JOIN Coincident C ON LD.date = C.date
WHERE LD.date >= '1980-07-01'
AND LD.date <= '1981-07-01'
"""


# Run the query
Pre_1981_Recession = pd.read_sql_query(sql_query, conn)

# Save to CSV
csv_filename = "Pre_1981_Recession.csv"
Pre_1981_Recession.to_csv(csv_filename, index=False)

# Close DB connection
conn.close()

# Show final DataFrame
Pre_1981_Recession


,Date,Fed_Funds_Rate_Change,Business_Sentiment_Growth_Rate,Consumer_Sentiment_Growth_Rate,Oil_Price_Change,Yield_Curve_Change,Housing_Starts_Change,Jobless_Claims_Growth_Rate,Job_Growth_Rate,Date,GDP_Growth_Rate,Change_in_CPI,Labor_Force_Change,Change_in_PPI,Unemployment_Rate_Change,Date,Capacity_Utilization_Growth_Rate,Industrial_Production_Growth_Rate
0,1980-07-01,-2.85,19.0,18.0,0.0,2.25,21.0,-11.0,-1.0,1980-07-01,0.0,1.7,0.0,2.5,0.9,1980-07-01,-2.17,-4.39
1,1980-10-01,6.03,-137.0,20.0,-9.0,-1.56,20.0,-20.0,1.0,1980-10-01,2.0,2.1,-0.1,2.5,-0.3,1980-10-01,3.26,3.29
2,1981-01-01,0.75,-115.0,-5.0,6.0,-0.35,2.0,-1.0,1.0,1981-01-01,2.0,2.5,0.2,2.4,0.0,1981-01-01,-0.40,1.69
3,1981-04-01,1.19,-300.0,1.0,0.0,0.22,-12.0,1.0,0.0,1981-04-01,-1.0,1.9,0.3,2.8,-0.3,1981-04-01,-0.36,-0.33
4,1981-07-01,-0.20,-306.0,2.0,-5.0,-0.60,-23.0,8.0,0.0,1981-07-01,1.0,2.4,-0.4,1.0,0.0,1981-07-01,0.22,1.75


### 1990 Recession (Start= July 1990)

In [20]:
import pandas as pd
import sqlite3

# Create in-memory SQLite connection
conn = sqlite3.connect(':memory:')

# Assume these DataFrames are already loaded:
# Final_Coincident_Indicators_df, Final_Leading_Indicators_df, Final_Lagging_Indicators_df

# Save them to the SQLite database
Final_Coincident_Indicators_df.to_sql('Coincident', conn, index=False, if_exists='replace')
Final_Leading_Indicators_df.to_sql('Leading', conn, index=False, if_exists='replace')
Final_Lagging_Indicators_df.to_sql('Lagging', conn, index=False, if_exists='replace')

# Proper SQL query
sql_query = """
SELECT *
FROM Leading LD
JOIN Lagging LG ON LD.date = LG.date
JOIN Coincident C ON LD.date = C.date
WHERE LD.date >= '1989-07-01'
AND LD.date <= '1990-07-01'
"""


# Run the query
Pre_1990_Recession = pd.read_sql_query(sql_query, conn)

# Save to CSV
csv_filename = "Pre_1990_Recession.csv"
Pre_1990_Recession.to_csv(csv_filename, index=False)

# Close DB connection
conn.close()

# Show final DataFrame
Pre_1990_Recession


,Date,Fed_Funds_Rate_Change,Business_Sentiment_Growth_Rate,Consumer_Sentiment_Growth_Rate,Oil_Price_Change,Yield_Curve_Change,Housing_Starts_Change,Jobless_Claims_Growth_Rate,Job_Growth_Rate,Date,GDP_Growth_Rate,Change_in_CPI,Labor_Force_Change,Change_in_PPI,Unemployment_Rate_Change,Date,Capacity_Utilization_Growth_Rate,Industrial_Production_Growth_Rate
0,1989-07-01,-0.65,-286.0,1.0,-7.0,0.48,7.0,5.0,0.0,1989-07-01,1.0,1.4,0.1,0.5,0.0,1989-07-01,-1.27,-1.51
1,1989-10-01,-0.47,-22.0,2.0,2.0,-0.17,-1.0,4.0,0.0,1989-10-01,0.0,0.9,0.0,0.0,0.1,1989-10-01,-0.34,0.51
2,1990-01-01,-0.36,-13.0,-1.0,13.0,0.09,10.0,1.0,1.0,1990-01-01,1.0,2.1,0.3,2.1,0.1,1990-01-01,0.00,0.38
3,1990-04-01,-0.01,-91.0,1.0,-18.0,-0.06,-20.0,2.0,0.0,1990-04-01,0.0,1.4,-0.2,-0.8,0.0,1990-04-01,0.03,1.16
4,1990-04-01,0.00,-100.0,0.0,0.0,0.00,0.0,0.0,0.0,1990-04-01,0.0,1.4,-0.2,-0.8,0.0,1990-04-01,0.03,1.16
5,1990-07-01,-0.08,-inf,-6.0,0.0,0.25,-6.0,6.0,0.0,1990-07-01,0.0,1.6,-0.1,0.4,0.1,1990-07-01,-0.16,0.47


### 2001 Recession (Start= March 2001)

In [21]:
import pandas as pd
import sqlite3

# Create in-memory SQLite connection
conn = sqlite3.connect(':memory:')

# Assume these DataFrames are already loaded:
# Final_Coincident_Indicators_df, Final_Leading_Indicators_df, Final_Lagging_Indicators_df

# Save them to the SQLite database
Final_Coincident_Indicators_df.to_sql('Coincident', conn, index=False, if_exists='replace')
Final_Leading_Indicators_df.to_sql('Leading', conn, index=False, if_exists='replace')
Final_Lagging_Indicators_df.to_sql('Lagging', conn, index=False, if_exists='replace')

# Proper SQL query
sql_query = """
SELECT *
FROM Leading LD
JOIN Lagging LG ON LD.date = LG.date
JOIN Coincident C ON LD.date = C.date
WHERE LD.date >= '2000-04-01'
AND LD.date <= '2001-04-01'
"""


# Run the query
Pre_2001_Recession = pd.read_sql_query(sql_query, conn)

# Save to CSV
csv_filename = "Pre_2001_Recession.csv"
Pre_2001_Recession.to_csv(csv_filename, index=False)

# Close DB connection
conn.close()

# Show final DataFrame
Pre_2001_Recession

,Date,Fed_Funds_Rate_Change,Business_Sentiment_Growth_Rate,Consumer_Sentiment_Growth_Rate,Oil_Price_Change,Yield_Curve_Change,Housing_Starts_Change,Jobless_Claims_Growth_Rate,Job_Growth_Rate,Date,GDP_Growth_Rate,Change_in_CPI,Labor_Force_Change,Change_in_PPI,Unemployment_Rate_Change,Date,Capacity_Utilization_Growth_Rate,Industrial_Production_Growth_Rate
0,2000-04-01,0.59,-30.0,-3.0,-5.0,-0.63,-1.0,-2.0,1.0,2000-04-01,2.0,1.6,0.0,2.4,-0.2,2000-04-01,0.21,1.37
1,2000-07-01,0.25,-47.0,-1.0,16.0,0.12,-10.0,8.0,0.0,2000-07-01,0.0,1.8,-0.4,3.0,0.2,2000-07-01,-1.06,0.18
2,2000-10-01,-0.05,-152.0,-2.0,11.0,0.12,6.0,9.0,0.0,2000-10-01,1.0,1.2,-0.1,1.7,-0.1,2000-10-01,-1.25,-0.21
3,2001-01-01,-0.87,562.0,-10.0,-11.0,0.57,3.0,12.0,0.0,2001-01-01,0.0,1.7,0.4,4.6,0.3,2001-01-01,-2.19,-0.81
4,2001-04-01,-1.27,-20.0,-7.0,-7.0,0.51,3.0,8.0,0.0,2001-04-01,1.0,0.8,-0.3,-3.6,0.2,2001-04-01,-2.16,-1.25


### 2008 Recession (Start= December 2007)

In [22]:
import pandas as pd
import sqlite3

# Create in-memory SQLite connection
conn = sqlite3.connect(':memory:')

# Assume these DataFrames are already loaded:
# Final_Coincident_Indicators_df, Final_Leading_Indicators_df, Final_Lagging_Indicators_df

# Save them to the SQLite database
Final_Coincident_Indicators_df.to_sql('Coincident', conn, index=False, if_exists='replace')
Final_Leading_Indicators_df.to_sql('Leading', conn, index=False, if_exists='replace')
Final_Lagging_Indicators_df.to_sql('Lagging', conn, index=False, if_exists='replace')

# Proper SQL query
sql_query = """
SELECT *
FROM Leading LD
JOIN Lagging LG ON LD.date = LG.date
JOIN Coincident C ON LD.date = C.date
WHERE LD.date >= '2007-01-01'
AND LD.date <= '2008-01-01'
"""


# Run the query
Pre_2008_Recession = pd.read_sql_query(sql_query, conn)

# Save to CSV
csv_filename = "Pre_2008_Recession.csv"
Pre_2008_Recession.to_csv(csv_filename, index=False)

# Close DB connection
conn.close()

# Show final DataFrame
Pre_2008_Recession

,Date,Fed_Funds_Rate_Change,Business_Sentiment_Growth_Rate,Consumer_Sentiment_Growth_Rate,Oil_Price_Change,Yield_Curve_Change,Housing_Starts_Change,Jobless_Claims_Growth_Rate,Job_Growth_Rate,Date,GDP_Growth_Rate,Change_in_CPI,Labor_Force_Change,Change_in_PPI,Unemployment_Rate_Change,Date,Capacity_Utilization_Growth_Rate,Industrial_Production_Growth_Rate
0,2007-01-01,0.01,-164.0,4.0,-7.0,-0.05,-5.0,-2.0,0.0,2007-01-01,0.0,1.54,0.2,1.8,0.2,2007-01-01,0.30,0.64
1,2007-04-01,0.00,-500.0,-10.0,17.0,0.15,6.0,-1.0,0.0,2007-04-01,1.0,2.47,-0.5,7.4,-0.1,2007-04-01,0.53,1.82
2,2007-07-01,-0.18,-18.0,4.0,16.0,0.16,-9.0,1.0,0.0,2007-07-01,1.0,1.70,0.1,3.7,0.2,2007-07-01,-0.29,-0.08
3,2007-10-01,-0.57,-83.0,-11.0,16.0,0.37,-7.0,7.0,0.0,2007-10-01,1.0,1.59,-0.2,-0.4,0.0,2007-10-01,0.27,0.15
4,2008-01-01,-1.32,75.0,-3.0,8.0,0.71,-14.0,4.0,0.0,2008-01-01,0.0,2.98,0.4,6.3,0.3,2008-01-01,-0.07,0.50


In [23]:
import pandas as pd
import sqlite3

# Create in-memory SQLite connection
conn = sqlite3.connect(':memory:')

# Assume these DataFrames are already loaded:
# Final_Coincident_Indicators_df, Final_Leading_Indicators_df, Final_Lagging_Indicators_df

# Save them to the SQLite database
Final_Coincident_Indicators_df.to_sql('Coincident', conn, index=False, if_exists='replace')
Final_Leading_Indicators_df.to_sql('Leading', conn, index=False, if_exists='replace')
Final_Lagging_Indicators_df.to_sql('Lagging', conn, index=False, if_exists='replace')

# Proper SQL query
sql_query = """
SELECT *
FROM Leading LD
JOIN Lagging LG ON LD.date = LG.date
JOIN Coincident C ON LD.date = C.date
WHERE LD.date >= '2019-04-01'
AND LD.date <= '2020-04-01'
"""


# Run the query
Pre_2020_Recession = pd.read_sql_query(sql_query, conn)

# Save to CSV
csv_filename = "Pre_2020_Recession.csv"
Pre_2020_Recession.to_csv(csv_filename, index=False)

# Close DB connection
conn.close()

# Show final DataFrame
Pre_2020_Recession

,Date,Fed_Funds_Rate_Change,Business_Sentiment_Growth_Rate,Consumer_Sentiment_Growth_Rate,Oil_Price_Change,Yield_Curve_Change,Housing_Starts_Change,Jobless_Claims_Growth_Rate,Job_Growth_Rate,Date,GDP_Growth_Rate,Change_in_CPI,Labor_Force_Change,Change_in_PPI,Unemployment_Rate_Change,Date,Capacity_Utilization_Growth_Rate,Industrial_Production_Growth_Rate
0,2019-04-01,0.00,-58.0,7.0,24.0,0.02,4.0,0.0,0.0,2019-04-01,1.0,2.67,-0.3,3.0,-0.3,2019-04-01,-0.86,-1.05
1,2019-07-01,-0.21,-57.0,1.0,-10.0,0.03,-3.0,-4.0,0.0,2019-07-01,1.0,0.57,0.3,-1.4,0.0,2019-07-01,-0.29,-0.26
2,2019-10-01,-0.54,-242.0,-3.0,-6.0,-0.06,7.0,7.0,0.0,2019-10-01,1.0,1.35,0.2,-2.1,-0.1,2019-10-01,-0.75,-0.41
3,2020-01-01,-0.40,-153.0,5.0,7.0,0.08,19.0,286.0,0.0,2020-01-01,-1.0,1.97,0.0,0.7,0.0,2020-01-01,-1.76,-0.25
4,2020-04-01,-1.19,-1044.0,-28.0,-71.0,0.19,-41.0,215.0,-14.0,2020-04-01,-8.0,-3.10,-3.2,-13.8,11.2,2020-04-01,-12.65,-16.44


| Indicator                         |   1980 |    1981 |    1990 |   2001 |   2008 |    2020 |
|:----------------------------------|-------:|--------:|--------:|-------:|-------:|--------:|
| Fed_Funds_Rate_Change             |   1    |    2.65 |    0.57 |  -1.86 |  -1.33 |   -1.19 |
| Business_Sentiment_Growth_Rate   | 295.84 | -325.3  | -inf    |  10.08 | 238.64 | -986.86 |
| Consumer_Sentiment_Growth_Rate   |  16.97 |  -15.87 |   -6.62 |  -4.15 |  -6.62 |  -34.64 |
| Oil_Price_Change                 |  12.07 |   -5.26 |    6.93 |  -2.04 |  15.15 |  -95.52 |
| Yield_Curve_Change               |   1.04 |   -2.85 |   -0.23 |   1.14 |   0.76 |    0.17 |
| Housing_Starts_Change            |  -2.51 |  -44.2  |  -12.74 |   3.67 |  -8.74 |  -44.75 |
| Jobless_Claims_Growth_Rate       |  -3.7  |   18.26 |    1.54 |   9.49 |   5.57 |  215.31 |
| Job_Growth_Rate                  |  -0.62 |    1.46 |   -0.13 |  -0.86 |  -0.31 |  -14.57 |
| GDP_Growth_Rate                  |   0.13 |    1.32 |   -0.67 |  -1.2  |  -0.73 |   -8.75 |
| Change_in_CPI                    |   1.4  |    0.7  |    0.2  |  -0.8  |   1.44 |   -5.77 |
| Labor_Force_Change               |   0    |   -0.4  |   -0.2  |  -0.3  |   0.2  |   -2.9  |
| Change_in_PPI                    |   1.1  |   -1.5  |   -0.1  |  -6    |   4.5  |  -16.8  |
| Unemployment_Rate_Change         |   0.2  |   -0.9  |    0.1  |   0.4  |   0.1  |   11.5  |
| Capacity_Utilization_Growth_Rate |   0.16 |    2.39 |    1.11 |  -2.37 |  -0.37 |  -11.79 |
| Industrial_Production_Growth_Rate|  -0.29 |    6.14 |    1.98 |  -2.62 |  -0.14 |  -15.39 |


Key Takeaways::
- **Interest Rates (Fed Funds):** Rose before early recessions (1980, 1981); fell into modern ones.
- **Yield Curve:** Flattens/inverts before recessions; steepens just before the downturn hits.
- **Business/Consumer Sentiment:** Dramatic collapses before 2001, 2008, 2020.
- **Housing Starts & Jobless Claims:** Early movers — major signals especially for 2008 and 2020.
- **Lagging Indicators (Unemployment, CPI, PPI):** Respond later, but accelerate into recessions.
- **2020 shows extreme outliers**: likely due to COVID shock rather than cyclical behavior

| Recession | Fed Funds | Yield Curve | Biz Sentiment | Cons. Sentiment | Housing Starts | Jobless Claims | Job Growth | GDP Growth | CPI | PPI | Unemp. Rate | Ind. Prod. | Cap. Util |
|-----------|--------------|----------------|------------------|--------------------|-------------------|-------------------|----------------|----------------|--------|--------|----------------|----------------|--------------|
| 1980      | 1.10         | -0.10          | -3.77            | -3.06              | -6.90             | 4.51              | 0.64           | 0.32           | 2.18   | 2.68   | 0.10           | 0.31           | -0.58         |
| 1981      | 0.98         | -0.01          | -167.68          | 7.51               | 1.36              | -4.61             | 0.17           | 0.83           | 2.12   | 2.24   | 0.06           | 0.40           | 0.11          |
| 1990      | -0.31        | 0.12           | 169.62              | -0.69              | -2.07             | 3.52              | 0.37           | 0.49           | 1.48   | 0.44   | 0.06           | 0.20           | -0.35         |
| 2001      | -0.27        | 0.14           | 62.62            | -4.55              | 0.32              | 6.85              | 0.22           | 0.56           | 1.42   | 1.62   | 0.08           | -0.14          | -1.29         |
| 2008      | -0.41        | 0.27           | -137.82          | -3.28              | -5.95             | 1.77              | 0.22           | 0.34           | 2.06   | 3.76   | 0.12           | 0.61           | 0.15          |
| 2020      | -0.47        | 0.05           | -310.75          | -3.74              | -2.77             | 100.85            | -2.58          | -1.32          | 0.69   | -2.72  | 2.16           | -3.68          | -3.26         |
